<h1><font color='blue'>Finding the Probability P(Y==1|X)</font></h1>

In [59]:
import sys
print(sys.executable)

from math import exp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm import tqdm
from sklearn.datasets import make_classification

from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV

C:\Users\magic\anaconda3\envs\ml1\python.exe


<h2><font color='Geen'> Task 1: Implementing Decision Function of SVM RBF Kernel</font></h2>

In [5]:
X, Y = make_classification(n_samples=5000, n_features=5, n_redundant=2,
                           n_classes=2, weights=[0.7], class_sep=0.7, random_state=15)

In [7]:
svc = SVC(gamma=0.001,C=100,kernel='rbf').fit(X,Y)

In [14]:
svc.decision_function(X)

5000

In [116]:
def dis_fn(X):
    gamma = 0.001
    g = -1*gamma
    vec = svc.support_vectors_
    coef = svc.dual_coef_
    # Y = list(svc.predict(X)[::-1]) # reverse
    result = []
    for q in tqdm(X):
        pt_sum = []
        for v in range(len(svc.support_vectors_)):
            norm = (np.linalg.norm(vec[v]-q)**2)
            kernel = coef[0][v]*exp(g*norm)
            pt_sum.append(kernel)
        result.append(sum(pt_sum)+svc.intercept_[0])
    # list comphrehension
    res = [sum([(coef[0][v]*exp(g*(np.linalg.norm(vec[v]-q)**2))) for v in range(len(svc.support_vectors_))])+svc.intercept_[0] for q in X]
    return result,res


In [117]:
res,_ = dis_fn(X)

100%|██████████| 5000/5000 [00:30<00:00, 165.86it/s]


In [126]:
res[:10]

[-3.923483789714974,
 2.4488237492535254,
 -3.2620508442473786,
 1.563765790696484,
 -2.743964668277935,
 2.4244769730037965,
 -1.3199795780550354,
 0.2925832541720914,
 -3.6810057283927713,
 -2.9499560810538363]

In [127]:
svc.decision_function(X[:10])

array([-3.92348379,  2.44882375, -3.26205084,  1.56376579, -2.74396467,
        2.42447697, -1.31997958,  0.29258325, -3.68100573, -2.94995608])

## I have achived same results

<h2><font color='Geen'> Task2: Implementing Platt Scaling to find P(Y==1|X)</font></h2>